주제 : 이진분류기법

목차

1. 이진분류기법이란?
2. 이진분류의 종류
3. 데이터 확인
4. EDA
5. 데이터 전처리 
6. 다양한 모델들을 통한 분류(예측)
7. 분류(예측) 결과 요약
8. 환경데이터로의 활용방안

1. 이진분류기법이란? 

- 분류는 어떠한 변수에 영향을 받는 결과를 연속적이지 않은 값들로 나눌 때 사용합니다. 특히나, 이진 분류는 구분하고자 하는 결과 값이 상호 배타적인 변수들 중 하나를 출력하는 분류 작업 유형입니다. 


2. 이진분류의 종류

 - Logistic Regression
 - Support Vector Machines(Linear and radial)
 - Random Forest
 - K-Nearest Neighbours
 - Naive Bayes
 - Decision Tree
 - Logistic Regression


3. 데이터 확인

-> 데이터 설명

- 해당 데이터는 1912년 4월 10일 영국의 사우샘프턴을 떠나 미국의 뉴욕으로 향하던 첫 항해 중에 4월 15일 빙산과 충돌한 후 침몰하여 2224명의 승객과 승무원 중 1502명이 사망했던 사건을 다룬 데이터입니다. 

-> 데이터 확인

In [1]:
# 라이브러리 불러오기 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

* 데이터를 불러오기 앞서 데이터 분석시에 사용하는 라이브러리들을 불러오겠습니다

In [2]:
# 데이터 불러오기

data = pd.read_csv('C:/Users/USER/Desktop/python 연습/data/titanic/train.csv')

* 우선적으로 trainning set을 먼저 불러와서 데이터를 확인해보도록 하겠습니다. 

In [3]:
# 데이터 내용 확인
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


데이터 안에는 다음과 변수들을 가지고 있습니다 


In [ ]:
# checking for total null values(결측값 찾기)

data.isnull().sum() 

In [ ]:
결측값을 가지고 있는 변수를 확인한 결과, Age(나이), Cabin(객실번호), Embarked(탑승 항구) 이었습니다. 

변수들의 유형 구분
1. 범주형 변수(categorical Feature)

- 범주형 변수는 두개 이상의 값들이 하나의 범주로 이루어진 변수입니다. 예를 들어, 성별은 두 개의 값(남성과 여성)을 갖는 범주형 변수입니다. 범주형 변수는 명목변수 중에 하나이기도 합니다. 

-> 해당 변수 : Sex(성별), Embarked(탑승 항구)

2. 서열형 변수(Ordinal Features)

- 서열형 변수는 범주형 변수와 유사하지만, 차이점은 값 사이의 상대적 순서나 정렬을 가질 수 있다는 것 입니다. 예를 들어, 만족도 조사시, 만족/보통/불만족 과 같은 값이 있다면 이 만족도는 서열형 변수가 됩니다. 

-> 해당 변수 : PClass(티켓등급)

3. 연속형 변수(Continous Features)

- 연속형 변수는 대상들에 대해 측정하면 대상들이 서로 연속된 값을 갖게하는 경우에 대한 변수를 말합니다. 예를 들어, 키와 몸무게는 대표적인 연속형 변수 중에 하나입니다.

-> 해당 변수 : Fare(탑승료), Age(나이)


4. EDA(각 변수들에 대한 탐색적 자료 분석)

In [ ]:
# 생존(Survived)
# 얼마나 많은 사람들이 살아남았을까요?

f, ax = plt.subplots(1,2, figsize = (18,8))
data['Survived'].value_counts().plot.pie(explode = [0,0.1], autopct = '%1.1f%%', ax = ax[0], shadow = True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data = data, ax = ax[1])
ax[1].set_title('Survived')
plt.show()

해당 사고에서 살아남은 승객은 절반에도 못 미치는 것이 분명히 보입니다. 

trainning set 승객 891명 중 350명 정도만 살아남았습니다. 즉, 전체 trainning set 중 38.4%만이 살아남았습니다. 

데이터로부터 더 나은 통찰력을 얻기 위해, 승객의 어떤 범주가 살아남았고 누가 살아남지 못했는지를 보기위해 더 깊게 파고들어보겠습니다.

먼저 승객들의 다양한 유형에 따른 특징을 살펴보겠습니다


In [ ]:
# 성별(Sex) --> 범주형 변수(Categorical Feature)
# 성별에 따른 생존율은 어땠을까?

data.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
f, ax = plt.subplots(1,2, figsize = (18,8))
data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax = ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue = 'Survived', data = data, ax = ax[1])
ax[1].set_title('Sex:Survived vs Dead')
plt.show()

우선 위의 표에서는 타이타닉에 타고 있는 남성들의 수는 여성들의 숫자보다 훨씬 많다는 것을 확인할 수 있습니다. 그리고 여성들의 생존자 수가 남성들의 생존자 수의 2배 정도였습니다. 아래 그래프에서 여성 승객의 생존율은 약 75%인데 반해 남성 승객은 약 18-19% 정도입니다. 

In [ ]:
# 티켓 등급(Pclass) --> 서열형 변수(Ordinal Feature)

pd.crosstab(data.Pclass, data.Survived,margins = True).style.background_gradient(cmap = 'summer_r')

In [ ]:
f, ax = plt.subplots(1,2, figsize = (18,8))
data['Pclass'].value_counts().plot.bar(color = ['#CD7F32', '#FFDF00', '#D3D3D3'], ax = ax[0])
ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue = 'Survived', data = data, ax = ax[1])
ax[1].set_title('Pclass:Survived vs Dead')

표와 그래프를 확인해보았을때, 1클래스의 승객들이 가장 높은 생존율을 보였습니다. 그리고 3클래스의 승객 수는 훨씬 더 높았지만, 생존율은 약 25%로 가장 낮았습니다. 

In [ ]:
# 성별(Sex)과 티켓등급(Pclass) 에 따른 생존율 분석

pd.crosstab([data.Sex, data.Survived], data.Pclass, margins = True).style.background_gradient(cmap = 'summer_r')

In [ ]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', data = data)
plt.show()

표와 factorplot을 확인한 결과, 1클래스의 여성 94명 중 3명만이 사망했기 때문에
1클래스의 여성승객 생존율은 약 95-96%라는 것을 쉽게 알 수 있습니다.
티켓등급과 관계없이, 여성 승객들은 구조하는 동안 우선권을 가졌던 것으로 보여집니다. 1클래스의 남성 승객들도 생존율이 매우 낮습니다.

In [ ]:
# 나이(Age) --> 연속형 변수(Continous Feature)

print('Oldest Passenger was of:', data['Age'].max(), 'Years')
print('Youngest Passenger was of:', data['Age'].min(), 'Years')
print('Average Age on the ship:', data['Age'].mean(), 'Years')

In [ ]:
f, ax = plt.subplots(1,2, figsize = (18,8))
sns.violinplot("Pclass", "Age", hue = "Survived", data = data, split = True, ax = ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex", "Age", hue = "Survived", data= data, split = True, ax = ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

우선 티켓등급에 따라 어린이의 수가 증가하고, 10세 미만(즉,어린이)의 경우 생존율이 티켓등급에 관계없이 양호해보입니다. 그리고 1클래스부터 20-50세의 승객은 생존 가능성이 높고 여성 승객은 더욱 높습니다. 남성 승객의 경우 나이가 많아지면 생존확률이 줄어듭니다. 

이전에도 살펴보았듯이, 나이 변수는 177개의 결측치를 가지고 있습니다. 이러한
결측치를 대체하기위해 데이터셋의 평균 연령을 할당하고자 합니다. 그런데, 문제는 나이차이가 많이 나는 사람들이 많았다는 것입니다. 4살일 수 있는 사람에게 평균값인 29살을 배치할 수는 없을 것입니다. 그래서, 해결책으로 사용할 수 있는 방법은 이름에서 부르는 호칭을 활용하는 방법입니다. 이름에 붙어있는 Mr, Mrs 등을 통해 나이대를 구분하고 그에 따라 나이대의 평균값을 할당할 것 입니다. 

In [ ]:
# Age 결측값 해결 -- "What's In A Name??" 

In [ ]:
data['Initial'] = 0
for i in data:
    data['Initial'] = data.Name.str.extract('([A-Za-z]+)\.')
    

In [ ]:
pd.crosstab(data.Initial, data.Sex).T.style.background_gradient(cmap = 'summer_r')

호칭들을 뽑아본 결과 총 16개의 호칭이 있었다는 것을 확인할 수 있습니다

In [ ]:
data['Initial'].replace(['Mlle', 'Mme', 'Ms','Dr','Major', 'Lady', 'Countess', 'Jonkheer', 'Col', 'Rev', 'Capt', 'Sir', 'Don'], ['Miss','Miss', 'Miss', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Other', 'Other', 'Other', 'Mr', 'Mr', 'Mr'], inplace = True)

In [ ]:
data.groupby('Initial')['Age'].mean()

호칭을 철자가 틀린 경우나 명확히 구분 지어서 수정해서 각 호칭별 평균값을 계산해보았습니다. Master 는 평균 4.5세, Miss는 평균 21.86세, Mr는 32.74세, Mrs는 35.98세, Other는 45.89세로 나타났습니다.

In [ ]:
# Age 결측값을 각 호칭별 평균 값으로 대체하기

data.loc[(data.Age.isnull())&(data.Initial == 'Mr'), 'Age'] = 33
data.loc[(data.Age.isnull())&(data.Initial == 'Mrs'),'Age'] = 36
data.loc[(data.Age.isnull())&(data.Initial == 'Master'), 'Age'] = 5
data.loc[(data.Age.isnull())&(data.Initial == 'Miss'), 'Age'] = 22
data.loc[(data.Age.isnull())&(data.Initial == 'Other'), 'Age'] = 46


In [ ]:
data.Age.isnull().any()

In [ ]:
f, ax= plt.subplots(1,2, figsize = (20,10))
data[data['Survived'] == 0].Age.plot.hist(ax = ax[0], bins = 20, edgecolor = 'black', color = 'red')
ax[0].set_title('Survived = 0')
x1 = list(range(0,85,5))
ax[0].set_xticks(x1)
data[data['Survived'] == 1].Age.plot.hist(ax = ax[1], color = 'green', bins = 20, edgecolor = 'black')
ax[1].set_title('Survived = 1')
x2 = list(range(0,85,5))
ax[1].set_xticks(x2)
plt.show()

결측값을 수정하고 생존자 수와 사망자 수의 분포를 확인한 결과, 우선 어린 아이들은 비교적 많이 구조 받았습니다. 그리고 가장 나이가 많으신(80세)분도 구조를 받았습니다. 최대 사망자 나이대는 30~40세 였습니다.

In [ ]:
sns.factorplot('Pclass', 'Survived', col = 'Initial', data = data)
plt.show()

호칭별로 살펴본 결과, 여성과 아이들에 대한 구조가 먼저 이뤄졌을 것이라는 것을 
유추해 볼 수 있습니다.

In [ ]:
# Embarked(탑승항구) --> 범주형 변수(Categorical Feature)

In [ ]:
pd.crosstab(data.Embarked, data.Survived, margins = True).style.background_gradient(cmap = 'summer_r')

In [ ]:
pd.crosstab([data.Embarked,data.Pclass], [data.Sex, data.Survived], margins = True).style.background_gradient(cmap = 'summer_r')

In [ ]:
sns.factorplot('Embarked', 'Survived', data = data)
fig = plt.gcf()
fig.set_size_inches(5,3)
plt.show()

In [ ]:
C항구의 생존 가능성은 0.55 전후로 가장 높은 반면 S항구의 경우는 가장 낮았습니다.

In [ ]:
f, ax = plt.subplots(2,2, figsize = (20,15))
sns.countplot('Embarked', data = data, ax = ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked', hue = 'Sex', data = data, ax = ax[0,1])
ax[0,1].set_title('Male - Female Split for Embarked')
sns.countplot('Embarked', hue = 'Survived', data = data, ax = ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked', hue = 'Pclass', data = data, ax = ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace = 0.2, hspace = 0.5)
plt.show()

탑승 항구에 따라서 다양한 특징을 살펴본 결과, S항구에서 탑승한 최다 승객들 중 대부분은 3클래스 티켓등급이었습니다.그리고 항구C에서 탑승한 승객들은 그들 중 상당한 비율이 살아남았습니다.상대적으로 1클래스와 2클래스 티켓등급 승객들이 많아서 그랬을 것으로 보입니다. Q항구는 승객의 95% 가까이가 3클래스 티켓등급이었습니다.

In [ ]:
sns.factorplot('Pclass', 'Survived', hue = 'Sex', col = 'Embarked', data = data)
plt.show()

탑승 항구별 생존율을 티켓등급별 남녀구분별로 살펴보았을때, 티켓등급에 관계없이 
클래스 1과 클래스 2의 경우 생존 가능성은 거의 1입니다. 그리고 S항구는 남녀 생존율이 전반적으로 낮았고 3클래스의 경우가 가장 심했습니다.Q항구는 거의 모두가 3클래스 티켓 등급이었습니다.

Embarked 결측값 해결 
위에서 S항구에서의 승객이 가장 많이 탑승한 것을 확인했기 떄문에 결측값을 S로 
교체하려고 합니다.

In [ ]:
data['Embarked'].fillna('S', inplace = True)

In [ ]:
data.Embarked.isnull().any()

In [ ]:
# 함께 탑승한 형제와 배우자의 수(SibSip) --> 이산형 변수(Discrete Feature)

pd. crosstab([data.SibSp], data.Survived).style.background_gradient(cmap = 'summer_r')

In [ ]:
f, ax = plt.subplots(1,2, figsize = (20,8))
sns.barplot('SibSp', 'Survived', data = data, ax = ax[0])
ax[0].set_title('SibSp vs Survived')
sns.factorplot('SibSp', 'Survived', data = data, ax = ax[1])
ax[1].set_title('SibSp vs Survived')
plt.close(2)
plt.show()

In [ ]:
pd.crosstab(data.SibSp, data.Pclass).style.background_gradient(cmap = 'summer_r')

막대 그래프와 factor 그래프를 보면, 만약 승객이 형제자매가 없는 경우라면, 생존율이 34.5%라는 것을 알 수 있습니다. 형제의 수가 증가하면 생존율은 감소 경향을 보이고 있습니다.만약에, 배 안에 형제가 있다면, 먼저 자신을 구하지 않고 형제를 먼저 구하려고 했을 수 있다는 것 입니다. 5-8의 구성을 가진 가정의 생존율은 0%입니다.
이 말인즉슨, 형제가 4명 이상인 사람이 3 클래스 티켓등급이라는 것을 표에서 확인할 수 있고 3 클래스의 대가족이 모두 죽었다는 것을 확인할 수 있습니다. 

In [ ]:
# 함께 탑승한 부모, 아이의 수(Parch) --> 이산형 변수(Discrete feature)

pd.crosstab(data.Parch, data.Pclass).style.background_gradient(cmap = 'summer_r')

표에서 보았을때, 더 큰 가족구성원들이 3 클래스에 있다는 것을 확인할 수 있습니다.

In [ ]:
f, ax = plt.subplots(1,2, figsize = (20,8))
sns.barplot('Parch', 'Survived', data = data, ax = ax[0])
ax[0].set_title('Parch vs Survived')
sns.factorplot('Parch', 'Survived', data = data, ax = ax[1])
ax[1].set_title('Parch vs Survived')
plt.close(2)
plt.show()

여기에서도 SibSp에서와 비슷한 결과를 보입니다. 부모를 태운 승객은 생존 가능성이 더 높습니다. 그러나 수가 증가함에 따라 감소합니다.
생존 가능성은 1-3명의 부모를 가진 사람에게 좋았습니다. 혼자라는 것 또한 생존 가능성이 적었으며 누군가가 배 안에 4명 이상의 구성원과 같이했다면 생존 가능성이 줄어든다는 것이 확인되었습니다.

In [ ]:
# 탑승료(Fare) --> 연속형 변수(Continous Feature)

print('Highest Fare was:', data['Fare'].max())
print('Lowest Fare was:', data['Fare'].min())
print('Average Fare was:', data['Fare'].mean())

In [ ]:
f, ax = plt.subplots(1,3, figsize = (20,8))
sns.distplot(data[data['Pclass'] == 1].Fare, ax = ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(data[data['Pclass'] == 2].Fare, ax = ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(data[data['Pclass'] == 3].Fare, ax = ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

탑승료의 분포를 확인해 보았을때, 1클래스의 티켓 등급의 승객 요금에는 많은 편차가 있을 것으로 보이며, 이 분포는 기준에 따라 차이가 커질 것으로 보입니다. 

각 변수들의 탐색적 자료 분석 요약

성별(Sex) = 남성에 비해 여성의 생존 가능성은 높습니다

티켓등급(Pclass) = 1 클래스 승객이 더 나은 생존 가능성이 있었다는 뚜렷한 추세가 있습니다. 3 클래스 승객은 생존율은 가장 낮았습니다. 여성 승객의 경우 1클래스에서 생존할 가능성은 거의 1이며 2클래스에서 생존할 가능성도 높았습니다

나이(Age) : 5-10세 미만의 어린이들은 생존할 가능성이 높았습니다. 15세에서 35세 사이의 승객들은 많이 사망하였습니다.

탑승항구(Embarked) : 대부분의 1클래스의 승객들이 Q항구였고, 승객들 중 대다수가 3클래스이었음에도 불구하고 C항구에서의 생존 가능성은 더 좋아보였습니다. 

함께 탄 가족구성원(Parch + SibSp) : 형제자매가 1-2명, 부모가 1-3명 있으면 혼자 있거나 가족이 함께 여행하는 것보다 진부함이 더 잘 나타납니다.

In [ ]:
# 각 변수별 상관관계 분석

sns.heatmap(data.corr(), annot = True, cmap = 'RdYlGn', linewidths = 0.2)
fig = plt.gcf()
fig.set_size_inches(10,10)
plt.show()

상관관계를 확인하는 이유는 거의 동일한 정보를 포함하는 두 개의 변수가 있다면 그 변수 중에 하나를 제거해줌으로써 모델의 정확도를 높일 수 있기 때문입니다. 이를 다중공산성(MultiColinearity) 문제라고 알려져있습니다. 

히트맵을 통한 각 변수간의 상관관계를 확인한 결과, 전체적으로 각 변수간에 상관관계는 그다지 높지 않다는 것을 확인할 수 있습니다. 가장 높은 상관관계는 SibSp와 Parch, 0.41이었습니다. 

5. 데이터 전처리

데이터 전처리는 여러 변수를 가지고 있는 데이터를 확인하였을때, 모든 변수가 중요하지 않을 것입니다. 그래서 제거해야할 중복변수가 많을 수 있습니다. 또한, 기존의 변수들에서 정보를 얻어서 새로운 형태의 변수를 추가할 수도 있습니다.예를 들어 Name(이름 변수)를 사용하여 호칭 변수를 얻는 것입니다.

- Age(나이) 변수

Age(나이)는 연속형 변수로써 값을 구분하는 것이 쉽지가 않습니다. 그래서 모델링시 구분을 좀 더 간편하게 하기위해 연속적인 값들을 범주형 값으로 변환하고자 합니다. 여기에서는 단순히 구간을 나누는 방법을 사용하고자 합니다. 승객의 최대 연령은 80세 였고, 따라서 0-80의 범위를 5개의 구간으로 나누고 16세마다의 구간을 형성하게 됩니다.

In [ ]:
data['Age_band']=0
data.loc[data['Age'] <= 16, 'Age_band'] = 0
data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age_band'] = 1
data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age_band'] = 2
data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age_band'] = 3
data.loc[data['Age'] > 64, 'Age_band'] = 4
data.head(2)

In [ ]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap = 'summer')

In [ ]:
sns.factorplot('Age_band', 'Survived', data=data, col = 'Pclass')
plt.show()

티켓등급을 가리지 않고 나이가 들수록 생존율이 감소한다는 것을 확인할 수 있습니다.

- Family_Size(가족의 수) and Alone(혼자 탄 경우)

함께 탄 사람을 확인할 수 있는 Parch변수와 SibSp변수를 합침으로써 가족구성원의 수와 혼자 탄 경우를 파악할 수 있습니다. 생존율이 승객의 가족 규모와 관련이 있는지 확인할 수 있도록 결합된 자료를 제공합니다. 

In [ ]:
data['Family_Size'] = 0
data['Family_Size'] = data['Parch'] + data['SibSp']
data['Alone'] = 0
data.loc[data.Family_Size == 0, 'Alone'] = 1

In [ ]:
f, ax = plt.subplots(1,2, figsize = (18,6))
sns.factorplot('Family_Size', 'Survived', data = data, ax = ax[0])
ax[0].set_title('Famliy_Size vs Survived')
sns.factorplot('Alone', 'Survived', data = data, ax = ax[1])
ax[1].set_title('Alone vs Survived')
plt.close(2)
plt.close(3)
plt.show()

Family_Size=0은 승객이 혼자라는 것을 의미합니다. 분명 혼자이면 생존 가능성은 매우 낮습니다. 가족규모가 4이상인 경우, 확률도 줄어듭니다. 

In [ ]:
sns.factorplot('Alone', 'Survived', data = data, hue = 'Sex', col = 'Pclass')
plt.show()

혼자 탄 경우는 3 클래스의 가족이 있는 경우를 제외하고는 성별이나 티켓등급에 상관없이 좋지 않다는 것을 알 수 있습니다.

- Fare(탑승료) 변수

탑승료 또한 연속적인 특징이기 때문에, 우리는 서열형 변수로 바꾸어 줄 수 있습니다. 

그래서 5개의 구간으로 탑승료를 나누고자합니다.

In [ ]:
# Fare_Range

data['Fare_Range'] = pd.qcut(data['Fare'], 4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap = 'summer_r')

In [ ]:
data['Fare_cat'] = 0
data.loc[data['Fare'] <= 7.91, 'Fare_cat'] = 0
data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare_cat'] = 1
data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare_cat'] = 2
data.loc[(data['Fare'] >31) & (data['Fare'] <=513), 'Fare_cat'] = 3

In [ ]:
sns.factorplot('Fare_cat', 'Survived', data = data, hue = 'Sex')
plt.show()

탑승료 구간이 증가할수록 생존 가능성은 증가하는 경향을 보이고 있습니다.

- 문자열 값을 숫자로 변환

머신러닝 모델링시 문자열 값들을 사용할 수 없기 때문에 성별(Sex),탑승항구(Embarked) 등의 변수들을 숫자값으로 변환할 필요가 있습니다. 

In [ ]:
data['Sex'].replace(['male', 'female'], [0,1], inplace = True)
data['Embarked'].replace(['S','C','Q'],[0,1,2], inplace = True)
data['Initial'].replace(['Mr', 'Mrs', 'Miss', 'Master', 'Other'], [0,1,2,3,4], inplace = True)

- 필요없는 변수 삭제

이름(Name) = 이름 변수는 어떤 범주형 값으로 변환이 불가하기에 필요하지 않습니다
나이(Age) = Age_band 변수를 가지고 있으므로 필요하지 않습니다
표(Ticket) = 분류할 수 없는 임의의 문자열입니다.
탑승료(Fare) = Fare_cat 변수가 있으므로 필요하지 않습니다
객실(Cabin) = 많은 결측치들과 많은 승객들이 여러 개의 객실을 가지고 있습니다. 그래서 도움이 되지 않는 변수입니다
Seagateld = 분류할 수 없습니다.

In [ ]:
data.drop(['Name', 'Age', 'Ticket', 'Fare', 'Cabin', 'Fare_Range', 'PassengerId'], axis = 1, inplace = True)
sns.heatmap(data.corr(), annot = True, cmap = 'RdYlGn', linewidths = 0.2, annot_kws = {'size' : 20})
fig = plt.gcf()
fig.set_size_inches(18, 15)
plt.xticks(fontsize = 14)
plt.yticks(fontsize = 14)
plt.show()

상관관계 히트맵에서 긍정적인 변수들을 확인할 수 있습니다. 그들 중 일부는 SibSp 와 Family_Size, Parch 와 Family_Size이고 일부는 Alone 와 Family_Size와 같은 부정적인 것입니다.

6. 다양한 모델들을 통한 분류(예측)

EDA를 통해서 인사이트를 얻었습니다. 하지만, 이 인사이트 만으로는 승객이 살아남을지 사망할지를 정확하게 예측할 수 없습니다. 그래서, 다양한 분류 알고리즘들을 통해 탑승객의 생존여부를 예측할 것 입니다.  

In [ ]:
# 모델링을 위한 라이브러리 불러오기

from sklearn.linear_model import LogisticRegression 
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
train, test = train_test_split(data, test_size = 0.3, random_state = 0, stratify = data['Survived'])
train_X = train[train.columns[1:]]
train_Y = train[train.columns[:1]]
test_X = test[test.columns[1:]]
test_Y = test[test.columns[:1]]
X = data[data.columns[1:]]
Y = data['Survived']

In [ ]:
# Radial Support Vector Machines(rbf-SVM)

model = svm.SVC(kernel = 'rbf', C = 1, gamma = 0.1)
model.fit(train_X, train_Y)
prediction1 = model.predict(test_X)
print('Accuracy for rbf SVM is', metrics.accuracy_score(prediction1, test_Y))

rbf-SVM을 통해 예측한 결과는 83.58%의 정확성을 나타냈습니다.

In [ ]:
# Linear Support Vector Machine(linear-SVM)

model = svm.SVC(kernel = 'linear', C = 0.1, gamma = 0.1)
model.fit(train_X, train_Y)
prediction2 = model.predict(test_X)
print('Accuracy for linear SVM is', metrics.accuracy_score(prediction2, test_Y))

linear-SVM을 통해 예측한 결과는 81.72%의 정확성을 나타냈습니다.

In [ ]:
# Logistic Regression

model = LogisticRegression()
model.fit(train_X, train_Y)
prediction3  = model.predict(test_X)
print('The accuracy of the Logistic Regression is', metrics.accuracy_score(prediction3, test_Y))

Logistic Regression을 통해 예측한 결과는 81.72%의 정확성을 나타냈습니다.

In [ ]:
# Decision Tree

model = DecisionTreeClassifier()
model.fit(train_X, train_Y)
prediction4 = model.predict(test_X)
print('The accuracy of the Decision Tree is', metrics.accuracy_score(prediction4, test_Y))

Decision Tree를 통해 예측한 결과는 80.60%의 정확성을 나타냈습니다.

In [ ]:
# K-Nearest Neighbors(KNN)

model = KNeighborsClassifier()
model.fit(train_X, train_Y)
prediction5 = model.predict(test_X)
print('The accuracy of the KNN is', metrics.accuracy_score(prediction5, test_Y))

K-Nearest Neighbors(KNN)을 통해 예측한 결과는 83.21%의 정확성을 나타냈습니다.

* KNN의 경우 n_neighbor 속성의 값을 변경함에 따라 정확도가 변경됩니다. n_neighbor의 다양한 값에 대한 정확도를 확인하겠습니다.

In [ ]:
a_index = list(range(1,11))
a = pd.Series()
x = [0,1,2,3,4,5,6,7,8,9,10]
for i in list(range(1,11)):
    model = KNeighborsClassifier(n_neighbors = i)
    model.fit(train_X, train_Y)
    prediction = model.predict(test_X)
    a = a.append(pd.Series(metrics.accuracy_score(prediction, test_Y)))
plt.plot(a_index, a)
plt.xticks(x)
fig = plt.gcf()
fig.set_size_inches(12,6)
plt.show()
print('Accuracies for different values of n are:', a.values, 'with the max value as', a.values.max())

정확도를 확인해보았을 때, n_neighbors의 값이 9일때, 정확도가 83.58%으로 가장 높은 것을 확인할 수 있습니다. 

In [ ]:
#Gaussian Naive Bayes

model = GaussianNB()
model.fit(train_X, train_Y)
prediction6 = model.predict(test_X)
print('The accuracy of the NaiveBayes is', metrics.accuracy_score(prediction6, test_Y))

Gaussian Naive Bayes을 통해 예측한 결과는 81.34%의 정확성을 나타냈습니다.

In [ ]:
# Random Forests

model = RandomForestClassifier(n_estimators = 100)
model.fit(train_X, train_Y)
prediction7 = model.predict(test_X)
print('The accuracy of the Random Forests is', metrics.accuracy_score(prediction7, test_Y))

Random Forests를 통해 예측한 결과는 82.09%의 정확성을 나타냈습니다.

# 모델 평가

- 모델이 얼마나 좋은 성능을 보일지 평가하는 방법을 말합니다. 모델 평가를 할 때는 trainning data set 뿐만 아니라 새로운 데이터가 들어왔을 때 잘 동작하는지를 측정하는데, 이를 일반화라고합니다. 일반화란 실제 머신러닝 시스템을 구축할 때 굉장히 요소인데, trainning data는 편향되어 있을 수가 있기 때문입니다. 모델의 복잡도를 조절하기위해 모델 평가에서는 정규화를 얼마나 강하게 해야할지와 모델이 관측되지 않은 데이터에 대해 잘 동작할지를 평가합니다.

* 교차 검증(Cross Validation)

- 주어진 데이터 전체를 사용해 모델을 만들 경우, 해당 데이터에는 잘 동작하지만 새로운 데이터에는 좋지 않은 성능을 보이는 모델을 만들 가능성이 있습니다. 이러한 현상이 발생하는 주요 이유 중 하나가 과적합입니다. 과적합 발생 여부를 알아내려면 주어진 데이터 중 일부는 모델을 만드는 trainning data로 사용하고, 나머지 일부는 test data로 사용해 모델을 평가해야 합니다.trainning data와 test data를 분리하여 모델을 만드는 방법 중 가장 자주 사용하는 기법으로, 데이터를 다수의 조각으로 나누어 훈련과 테스트를 반복하는 기법입니다. 

--> 해당 보고서에서는 K-Fold Cross Validation을 통해 모델을 평가해보겠습니다.

* K-Fold Cross Validation

- 데이터를 K개의 그룹으로 분할한 후 특정 하나의 그룹만을 제외하고 나머지 데이터를 trainning data로 사용합니다. 사용하지 않은 특정 하나의 그룹의 데이터를 가지고 테스트를 실행합니다. 그리고 해당 모델의 성능을 측정합니다. 평가는 테스트 데이터에서 얻은 정밀도의 평균과 표준편차를 구하는 것을 평가할 수 있습니다. 이렇게 K개의 그룹의 모든 데이터 셋을 같은 방식으로 반복합니다. 최종 성능은 이렇게해서 나온 결과들의 산술 평균으로 정할 수 있습니다. 보통 K의 값은 5~10으로 설정할 때가 많습니다. 이 검증은 K값이 커질수록 계산하는 연산이 많아져 속도가 느릴 수 있습니다. 

In [ ]:
# 교차검증 라이브러리 불러오기
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

In [ ]:
kfold = KFold(n_splits = 10, random_state = 22)
xyz = []
accuracy = []
std = []
classifiers = ['Linear Svm', 'Radial Svm', 'Logistic Regression', 'KNN', 'Decision Tree', 'Naive Bayes', 'Random Forest']
models = [svm.SVC(kernel = 'linear'), svm.SVC(kernel = 'rbf'), LogisticRegression(), KNeighborsClassifier(n_neighbors = 9), DecisionTreeClassifier(), GaussianNB(),RandomForestClassifier(n_estimators = 100)]
for i in models:
    model = i
    cv_result = cross_val_score(model, X, Y, cv = kfold, scoring = "accuracy")
    cv_result = cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2 = pd.DataFrame({'CV Mean':xyz, 'Std':std}, index = classifiers)
new_models_dataframe2

In [ ]:
plt.subplots(figsize = (12,6))
box = pd.DataFrame(accuracy, index = [classifiers])
box.T.boxplot()

In [ ]:
new_models_dataframe2['CV Mean'].plot.barh(width = 0.8)
plt.title('Average CV Mean Accuracy')
fig = plt.gcf()
fig.set_size_inches(8,5)
plt.show()

분류 모델들의 정확도는 불균형으로 인해 때때로 오해의 소지가 있을 수 있습니다. 모델이 잘못되었는지 또는 모델이 잘못 예측한 클래스를 보여주는 confusion matrix의 도움을 받아 요약된 결과를 얻을 수 있습니다. 

* Confusion Matrix(분류결과표)

분류결과표는 분류가 Y,N 두 종류가 있다고 할 때 분류 모델에서의 모델 평가 메트릭은 모델에서 구한 분류의 예측값과 데이터의 실제 분류인 실제 값의 발생빈도를 나열한 것 입니다.


In [ ]:
f, ax = plt.subplots(3,3,figsize = (12,10))

y_pred = cross_val_predict(svm.SVC(kernel='rbf'), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[0,0], annot = True, fmt = '2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')

y_pred = cross_val_predict(svm.SVC(kernel='linear'), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[0,1], annot = True, fmt = '2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')

y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors = 9), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[0,2], annot = True, fmt = '2.0f')
ax[0,2].set_title('Matrix for KNN')

y_pred = cross_val_predict(RandomForestClassifier(n_estimators = 100), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[1,0], annot = True, fmt = '2.0f')
ax[1,0].set_title('Matrix for Random-Forests')

y_pred = cross_val_predict(LogisticRegression(), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[1,1], annot = True, fmt = '2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')

y_pred = cross_val_predict(DecisionTreeClassifier(), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[1,2], annot = True, fmt = '2.0f')
ax[1,2].set_title('Matrix for Decision Tree')

y_pred = cross_val_predict(GaussianNB(), X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, y_pred), ax = ax[2,0], annot = True, fmt = '2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')

plt.subplots_adjust(hspace = 0.2, wspace = 0.2)
plt.show()

왼쪽 대각선은 각 등급에 대해 이루어진 정확한 예측의 수를 나타내고 오른쪽 대각선은 잘못된 예측의 수를 나타냅니다. 

모든 매트릭스를 살펴보았을때, rbf-SVM이 사망한 승객들을 정확하게 예측할 수 있는 더 높은 가능성을 가졌다고 할 수 있고, NaiveBayes는 생존한 승객들을 정확하게 예측할 수 있는 더 높은 가능성을 가지고 있습니다. 

* Hyper-Parameters Tunning

- 알고리즘의 성능을 결정하는 파라미터들을 최적화하는 과정입니다. 

In [ ]:
#SVM

from sklearn.model_selection import GridSearchCV
C=[0.05, 0.1, 0.2, 0.3, 0.25, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
gamma = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
kernel = ['rbf', 'linear']
hyper = {'kernel':kernel, 'C':C, 'gamma':gamma}
gd = GridSearchCV(estimator = svm.SVC(), param_grid = hyper, verbose = True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

SVM의 튜닝결과 정확성이 82.83%였고, 속도는 22.5s 였습니다.

In [ ]:
# Random Forests

n_estimators = range(100,1000,100)
hyper = {'n_estimators':n_estimators}
gd = GridSearchCV(estimator = RandomForestClassifier(random_state = 0), param_grid = hyper, verbose = True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

Random Forests의 튜닝결과 정확성이 81.71%였고, 속도는 49.1s 였습니다.

-> Ensembling 

앙상블은 모델의 정확성이나 성능을 높이는 좋은 방법입니다. 간단히 말해서, 강력한 단일 모델을 만들기 위한 다양한 간단한 모델의 조합입니다. 

* 종류

- Voting Classifier

- Bagging

- Boosting

* Voting Classifier

간단한 기계 학습 모델들의 예측을 결합하는 가장 간단한 방법입니다. 모든 하위 모델의 예측을 바탕으로 평균적인 예측 결과를 제공합니다. 하위 모델이나 기본 모델들은 모두 다 다른 유형입니다. 

In [ ]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf = VotingClassifier(estimators = [('KNN', KNeighborsClassifier(n_neighbors = 10)), ('RBF', svm.SVC(probability = True, kernel = 'rbf', C = 0.5, gamma = 0.1)), ('RFor', RandomForestClassifier(n_estimators = 500, random_state = 0)),('LR', LogisticRegression(C = 0.05)), ('DT', DecisionTreeClassifier(random_state = 0)),('NB', GaussianNB()),('svm', svm.SVC(kernel = 'linear', probability = True))
                                                 ], voting = 'soft').fit(train_X, train_Y)
print('The accuracy for ensembled model is:', ensemble_lin_rbf.score(test_X, test_Y))
cross = cross_val_score(ensemble_lin_rbf, X, Y, cv = 10, scoring = "accuracy")
print('The cross validated score is', cross.mean())

앙상블링 한 결과 82.46%의 정확도를 보였고, 교차 검증한 결과는 약 0.8238점 이었습니다.

* Bagging Classifier

데이터 세트의 작은 부분에 유사한 분류 모델을 적용한 다음 모든 예측의 평균을 취함으로써 작동합니다. 평균화로 인해, 분산이 감소합니다. Voting Classifier와 다르게 유사한 classifier을 사용합니다. 

In [ ]:
# Bagged KNN
from sklearn.ensemble import BaggingClassifier
model = BaggingClassifier(base_estimator = KNeighborsClassifier(n_neighbors = 3), random_state = 0, n_estimators = 700)
model.fit(train_X, train_Y)
prediction = model.predict(test_X)
print('The accuracy for bagged KNN is:', metrics.accuracy_score(prediction, test_Y))
result = cross_val_score(model, X, Y, cv = 10, scoring = 'accuracy')
print('The cross validated score for bagged KNN is:', result.mean())


KNN을 배깅한 결과, 정확도는 83.58%이고 교차 검증한 결과 약 0.8149점 이었습니다.

In [ ]:
#Bagged DecisionTree

model = BaggingClassifier(base_estimator = DecisionTreeClassifier(), random_state = 0, n_estimators = 100)
model.fit(train_X, train_Y)
prediction = model.predict(test_X)
print('The accuracy for bagged Decision Tree is:', metrics.accuracy_score(prediction, test_Y))
result = cross_val_score(model, X, Y, cv = 10, scoring = 'accuracy')
print('The cross validated score for bagged Decsion Tree is:', result.mean())

DecisionTree를 배깅한 결과, 정확도는 82.46%이고 교차 검증한 결과 약 0.8205점 이었습니다.

* Boosting

부스팅은 약한 모델을 결합해서 보다 정확하고 강력한 모델을 만드는 방식을 의미합니다 정확도가 낮더라고 일단 모델을 만들고, 드러난 약점(예측 오류)은 두 번째 모델이 보완합니다. 이 둘을 합치면 처음보다는 정확한 모델이 만들어지고, 그럼에도 여전히 남아 있는 문제는 다음 모델에서 보완하여 계속 더하는 과정을 반복하는 원리입니다. 

- AdaBoost(Adaptive Boosting)

Adaboost는 가중치를 부여한 약한 분류모델을 모아서 최종적인 강력한 분류모델을 생성하는 기법입니다. 다음 학습 과정에서 각각의 샘플들에 대해 가중치를 두고 모델링하여 다음 학습시 이전에 처리하지 못한 샘플들을 적응하게 더 잘 풀도록 개선하는 특성이 있습니다.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators = 200, random_state = 0, learning_rate = 0.1)
result = cross_val_score(ada, X, Y, cv = 10, scoring = 'accuracy')
print('The cross validated score for AdaBoost is:', result.mean())

In [ ]:
Adaboosting의 교차 검증 결과, 약 0.8250점 이었습니다.

- Gradient Boosting

Gradient Boosting은 이전 약한 분류 모델에서 발생된 잔차에 대해 다음의 약한 분류 모델을 적합시키는 것이 특징입니다. 

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
grad = GradientBoostingClassifier(n_estimators = 500, random_state = 0, learning_rate = 0.1)
result = cross_val_score(grad, X, Y, cv = 10, scoring = 'accuracy')
print('The cross validated score for Gradient Boosting is:', result.mean())

Gradientboosting의 교차 검증 결과, 약 0.8183점 이었습니다.

-XG Boost

XGBoost는 여러 개의 Decision Tree를 조합해서 사용하는 Ensemble 알고리즘입니다. 

In [ ]:
import xgboost as xg
xgboost = xg.XGBClassifier(n_estimators = 900, learning_rate = 0.1)
result = cross_val_score(xgboost, X, Y, cv = 10, scoring = 'accuracy')
print('The cross validated score for XGBoost is:', result.mean())

XGBoost의 교차 검증 결과, 약 0.8105점 이었습니다.

* 위 3가지 부스팅 검정 결과 AdaBoost가 가장 높은 정확도를 보였습니다. 그래서 하이퍼파라미터 튜닝을 통해 정확도와 성능을 높여보도록 하겠습니다.

In [ ]:
# Hyper-Parameter Tunning for AdaBoost

n_estimators = list(range(100,1100,100))
learn_rate = [0.05, 0.1, 0.2, 0.3, 0.25, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
hyper = {'n_estimators':n_estimators, 'learning_rate':learn_rate}
gd = GridSearchCV(estimator = AdaBoostClassifier(), param_grid = hyper, verbose = True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

하이퍼파라미터 튜닝 결과, n_estimators = 200이고 learning_rate가 0.05 였고 83.16%의 정확도를 보였습니다. 

In [ ]:
# Confusion Matrix for the Best Model

ada = AdaBoostClassifier(n_estimators = 200, random_state = 0, learning_rate = 0.05)
result = cross_val_predict(ada, X, Y, cv = 10)
sns.heatmap(confusion_matrix(Y, result), cmap = 'winter', annot = True, fmt = '2.0f')
plt.show()

AdaBoost의 492명의 사망자를 예측하였고, 249명의 생존자를 예측하였습니다. 

변수의 중요도

Randomforests, AdaBoost, Gradient Boosting, XgBoost 들이 분류를 하는데 있어 가장 중요한 영향을 끼친 변수들을 확인해보겠습니다.

In [ ]:
# Feature Importance 
f, ax = plt.subplots(2,2, figsize = (15,12))
model = RandomForestClassifier(n_estimators = 500, random_state = 0)
model.fit(X,Y)
pd.Series(model.feature_importances_, X.columns).sort_values(ascending = True).plot.barh(width = 0.08, ax = ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')
model = AdaBoostClassifier(n_estimators = 200, learning_rate = 0.05, random_state = 0)
model.fit(X, Y)
pd.Series(model.feature_importances_, X.columns).sort_values(ascending = True).plot.barh(width = 0.08, ax = ax[0, 1], color = '#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')
model = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.1, random_state = 0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending = True).plot.barh(width = 0.8, ax = ax[1,0], cmap = 'RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')
model = xg.XGBClassifier(n_estimators = 900, learning_rate = 0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending = True).plot.barh(width = 0.8, ax = ax[1,1], color = '#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')
plt.show()

일반적으로 중요한 변수로는 Initial(호칭), Fare_cat(탑승료 구간), Pclass(티켓등급), Family_Size(가족구성원수)등이 있었습니다. 그리고 Sex(성별)은 앞서 EDA를 통해서 티켓등급과 같이 확인하였을때 중요 요인으로 보여질 것으로 보였으나 Random Forests에서만 상대적으로 중요한 것으로 보여졌습니다. 그런데, 모든 분류기에서 가장 상위에 있는 호칭의 경우 이전 EDA를 성별과 양의 상관관계를 보였기 때문에 성별을 결코 간과할 수는 없다는 점을 확인할 수 있습니다.

7. 분류(예측) 결과 요약

- 튜닝전에는 rbf-svm과 KNN(n=9) 모델이 83.58%로 이 타이타닉 데이터의 결과를 가장 잘 분류하였습니다.

- 교차 검증을 통해 모델을 평가해보았을때, rbf-SVM이 CV Mean이 0.828290으로 가장 높았고 그 다음으로는 Random Forest가 0.822684였습니다.

- 혼동행렬을 통한 모델을 평가해보았을때는 rbf-SVM이 사망한 승객들을 정확하게 예측할 수 있는 더 높은 가능성을 가졌다고 할 수 있고, NaiveBayes는 생존한 승객들을 정확하게 예측할 수 있는 더 높은 가능성을 가지고 있습니다. 

- 상대적으로 정확도가 높았던 SVM과 Random Forests를 튜닝을 하여서 비교해본 결과 SVM이 82.82%로 81,8%의 정확도인 Random Forest보다 높게 나타났습니다. 

- 앙상블 기법을 통해서 확인보았을때는, Voting Classifier와 Bagging, Boosting의 종류에서 AdaBoost,Gradient Boost, XGBoost가 있었는데, 이중에서는 AdaBoost가 가장 높은 정확도를 나타낸 것을 확인할 수가 있었습니다. 

- 그리고 중요한 변수로는  Initial(호칭), Fare_cat(탑승료 구간), Pclass(티켓등급), Family_Size(가족구성원수)등이 있었습니다.

* 결론: Titanic 데이터를 다양한 이진분류기법들을 통해 생존자를 분류하고자 할 때, rbf-SVM 분류모델을 통해 생존자를 분류해내고자하고 분류를 하는데 있어 중요한 변수는 Initial(호칭), Fare_cat(탑승료 구간), Pclass(티켓등급), Family_Size(가족구성원수) 였습니다. 위 모델은 83%대의 높은 정확도로 분류(예측)을 할 수 있습니다. 
